In [2]:
# LIBRARIES
import numpy as np

import sys
sys.path.append('lib/')
from pca import *

In [37]:
def practical_qr_algorithm(matrix,k_max,zero_cutoff):
    m = matrix.shape[0]
    A = np.ones((k_max,m,m))
    A[0] = hessenberg(matrix)
    shift = np.ones(k_max)
    
    for k in range(1,k_max):
        # print('k:',k)
        shift[k] = A[k-1,m-1,m-1]
        Q,R = householder(A[k-1,:,:]-shift[k]*np.identity(m))
        A[k,:,:] = np.matmul(R,Q)+shift[k]*np.identity(m)
        for j in range(m-1):
            # print('  j:',j)
            if A[k,j,j+1] < zero_cutoff:
                A[k,j,j+1]=0
                A[k,j+1,j]=0
                # print('shift:',shift[k],',',A[k,j,j+1])
                # break
                A_1 = A[k,:j+1,:j+1]
                A_2 = A[k,j+1:,j+1:]
                eig=[shift[k]]
                if A_1.shape[0]>0: 
                    e_temp = practical_qr_algorithm(A_1,k_max,zero_cutoff)
                    # print('eig:',eig,'etemp:',e_temp)
                    if np.abs(e_temp)[0]>0:eig=eig+e_temp
                if A_2.shape[0]>0: 
                    e_temp = practical_qr_algorithm(A_2,k_max,zero_cutoff)
                    # print('eig:',eig,'etemp:',e_temp)
                    if np.abs(e_temp)[0]>0:eig=eig+e_temp
                return(eig)
        
    # return(shift[k])
    return(0)

                

In [38]:
A = np.array(
    [[1,2,3,4],
    [2,1,2,2],
    [3,2,1,3],
    [4,2,3,1]]
)

# A[0,0:A.shape[0]]
print(practical_qr_algorithm(A,10,1e-12))
print(np.linalg.eig(A)[0])

# j=2
# print(A[:j,:j])
# print(A[j:,j:])

IndexError: invalid index to scalar variable.